# Setup and Visualization of CO₂ Crystal Structure

This notebook provides a workflow for visualizing the crystal structure of carbon dioxide (CO₂). It includes steps for loading structural data, rendering 3D visualizations, and analyzing key features of the CO₂ crystal. Use this notebook to explore the arrangement of atoms and understand the structural properties of CO₂ in its solid state.

In [ ]:
# Standard Scientific Computing Libraries
import numpy as np              # Numerical arrays and mathematical operations
import pandas as pd             # Data manipulation and analysis (for tabular data)
import matplotlib.pyplot as plt # 2D plotting and visualization
import scipy as sp              # Scientific computing functions
from scipy.optimize import minimize  # Optimization routines
""
# System and File Handling
import os                       # Operating system interface
import sys                      # System-specific parameters and functions
import json                     # JSON data handling
import pickle                   # Python object serialization
from pathlib import Path        # Modern path handling
import warnings                 # Warning control
warnings.filterwarnings('ignore')  # Suppress minor warnings for cleaner output

# Crystal Structure and Materials Science
try:
    from pymatgen.core import Structure, Lattice, Element  # Core crystal structure classes
    from pymatgen.io.cif import CifWriter, CifParser       # CIF file I/O
    from pymatgen.symmetry.analyzer import SpacegroupAnalyzer  # Symmetry analysis
    from pymatgen.analysis.structure_matcher import StructureMatcher  # Structure comparison
    from pymatgen.transformations.standard_transformations import SupercellTransformation
    print("✓ PyMatGen successfully imported")
except ImportError:
    print("⚠️ PyMatGen not found. Install with: pip install pymatgen")

# 3D Visualization Libraries
try:
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D  # 3D plotting capabilities
    print("✓ Matplotlib 3D available")
except ImportError:
    print("⚠️ Matplotlib 3D not available")

# ASE (Atomic Simulation Environment) - Alternative structure handling
try:
    import ase                           # Atomic simulation environment
    from ase import Atoms                # Atomic structure representation
    from ase.io import read, write       # Structure I/O
    from ase.visualize import view       # Structure visualization
    from ase.build import bulk           # Structure building utilities
    print("✓ ASE successfully imported")
except ImportError:
    print("⚠️ ASE not found. Install with: pip install ase")

# Interactive 3D Visualization (Jupyter-specific)
try:
    import nglview as nv                 # Interactive 3D molecular visualization
    print("✓ NGLView available for interactive 3D visualization")
except ImportError:
    print("⚠️ NGLView not found. Install with: pip install nglview")

try:
    import py3Dmol                       # Another 3D molecular viewer
    print("✓ py3Dmol available for 3D visualization")
except ImportError:
    print("⚠️ py3Dmol not found. Install with: pip install py3dmol")

# Crystallographic and Symmetry Libraries
try:
    import spglib                        # Space group operations
    print("✓ Spglib available for symmetry operations")
except ImportError:
    print("⚠️ Spglib not found. Install with: pip install spglib")

# Custom Local Modules (your CO₂-specific code)
try:
    from spacegroup import Pa3, Cmce, P42mnm, R3c  # Custom space group classes
    from symmetry import build_unit_cell             # Custom structure building
    from energy import compute_energy_from_cell     # Custom energy calculations
    from minimize import objective_zerok         # Custom optimization
    from co2_potential import p1b, p2b             # CO₂ potential functions
    print("✓ Custom CO₂ modules successfully imported")
except ImportError as e:
    print(f"⚠️ Some custom modules not found: {e}")
    print("Make sure your custom .py files are in the same directory or Python path")

# Plotting Configuration
plt.style.use('default')  # Use default matplotlib style
plt.rcParams['figure.figsize'] = (12, 8)  # Default figure size
plt.rcParams['font.size'] = 11             # Default font size
plt.rcParams['axes.grid'] = True           # Enable grid by default
plt.rcParams['grid.alpha'] = 0.3           # Grid transparency

# Jupyter Notebook Configuration
try:
    from IPython.display import display, HTML, Image  # Enhanced display capabilities
    import ipywidgets as widgets                       # Interactive widgets
    print("✓ Jupyter/IPython enhancements available")
except ImportError:
    print("⚠️ Some Jupyter features may not be available")

print("\n" + "="*60)
print("SETUP COMPLETE")
print("="*60)
print("Available capabilities:")
print("  ✓ Numerical computing (NumPy, SciPy)")
print("  ✓ Data analysis (Pandas)")
print("  ✓ 2D/3D plotting (Matplotlib)")
if 'Structure' in globals():
    print("  ✓ Crystal structure handling (PyMatGen)")
if 'ase' in globals():
    print("  ✓ Alternative structure tools (ASE)")
if 'nv' in globals():
    print("  ✓ Interactive 3D visualization (NGLView)")
if 'py3Dmol' in globals():
    print("  ✓ Web-based 3D visualization (py3Dmol)")
print("\nReady for CO₂ crystal structure analysis!")

Now that we have verified the required libraries for our solid-state calculations, let's focus on the modules essential for constructing CO₂ phase diagrams.

The statement `from symmetry import build_unit_cell` imports the `build_unit_cell` function from the local `symmetry` module. This function is responsible for assembling the unit cell of a crystal structure using symmetry information.

The line `from spacegroup import Pa3, Cmce, P42mnm, R3c` imports four classes (`Pa3`, `Cmce`, `P42mnm`, and `R3c`) from the `spacegroup` module. Each class corresponds to a specific crystallographic space group, which defines the symmetry and atomic arrangement in different solid phases

In [ ]:
from symmetry import build_unit_cell
from spacegroup import Pa3, Cmce, P42mnm, R3c

Next, let's initialize the Pa3 (Phase I) structure of solid CO₂ as an example. To construct the cubic unit cell for this phase, only two parameters are required:

- `lattice_const`: Defines the size of the cubic cell.
- `bond_length`: Sets the distance between carbon and oxygen atoms within each CO₂ molecule.

With these two variables, the full crystal structure can be generated and visualized.

After specifying these two variables in units of angstroms, we can create a new class with these variables: `structure = Pa3(a=lattice_const)`

In [ ]:
bond = 1.16
a = 5.0

structure = Pa3(a, bond)  # Initialize the Pa3 structure with the given lattice constant

# You can also adjust the fractional coordinates 
# (bond length in this case) if needed
# structure.adjust_fractional_coords(bond)

pa3 = build_unit_cell(structure)

print("Pa3 structure built with bond length:", bond, "and lattice constant:", a)

Now that we have successfully initialized the Pa3 structure, let's create an enhanced 3D visualization that properly handles periodic boundary conditions and molecular bonding across unit cell boundaries.

The `plot3d_with_bonds()` function visualizes the atoms and bonds between atoms in the periodic system however at the edge of the supercell ecules span across periodic boundaries. This function creates a supercell to provide complete molecular representations and applies multiple visualization techniques to clearly show the atomic arrangement and bonding in the CO2 crystal structure.

In [ ]:
def plot3d_with_bonds(structure, spacefill=True, show_axes=True, bond_cutoff=2.0):
    """
    Enhanced 3D plotting with proper bond wrapping around unit cell boundaries.
    
    Parameters:
    -----------
    structure : pymatgen.Structure
        The crystal structure to visualize
    spacefill : bool
        Whether to use spacefill representation
    show_axes : bool
        Whether to show coordinate axes
    bond_cutoff : float
        Maximum distance to consider for bonding (in Angstroms)
    """

    # Extend the structure to include periodic images
    # This creates a 2x2x2 supercell to ensure proper bond visualization
    supercell_matrix = [[2, 0, 0], [0, 2, 0], [0, 0, 2]]
    
    # Create supercell transformation
    from pymatgen.transformations.standard_transformations import SupercellTransformation
    supercell_transform = SupercellTransformation(supercell_matrix)
    supercell_structure = supercell_transform.apply_transformation(structure)
    
    # Now create the NGLView visualization
    view = nv.show_pymatgen(supercell_structure)
    
    # Clear default representations
    view.clear_representations()
    
    # Add unit cell for the original cell (not the supercell)
    view.add_unitcell()
    #view = nv.show_pymatgen(structure)
    
    if spacefill:
        # Add spacefill representation
        view.add_spacefill(radius_type='vdw', radius=0.4, color_scheme='element')
        # Also add thin bonds to show connectivity
        view.add_ball_and_stick(radius=0.1, bond_radius=0.05)
    else:
        # Add ball and stick with proper bond detection
        view.add_ball_and_stick(radius=0.3, bond_radius=1)
    
    # Add distance-based bonds (this helps with periodic bonds)
    view.add_distance(atom_pair=[['C', 'O']], 
                     label_visible=True, 
                     color='gray',
                     radius=0.05)
    
    if show_axes:
        # Add coordinate axes
        view.shape.add_arrow([0, 0, 0], [2, 0, 0], [1, 0, 0], 0.3, "x-axis")
        view.shape.add_arrow([0, 0, 0], [0, 2, 0], [0, 1, 0], 0.3, "y-axis")
        view.shape.add_arrow([0, 0, 0], [0, 0, 2], [0, 0, 1], 0.3, "z-axis")
    
    view.camera = "perspective"
    return view

plot3d_with_bonds(pa3, spacefill=True, show_axes=True, bond_cutoff=2.0)

In [ ]:
def plot3d_with_wrapped_molecules(structure, spacefill=True, show_axes=True):
    """
    Enhanced 3D plotting with molecules wrapped to be intact using the custom wrapping function.
    
    Parameters:
    -----------
    structure : pymatgen.Structure
        The crystal structure to visualize
    spacefill : bool
        Whether to use spacefill representation
    show_axes : bool
        Whether to show coordinate axes
    """
    from symmetry import wrap_coordinates_by_carbon_fractional
    from pymatgen.core import Structure
    
    # Use the custom wrapping function to get intact molecules
    print("Wrapping molecules to ensure CO₂ integrity...")
    wrapped_cart_coords, molecule_assignment, molecules_grouped = wrap_coordinates_by_carbon_fractional(structure)
    
    # Create a new structure with the wrapped coordinates
    species = [str(site.specie) for site in structure]
    wrapped_structure = Structure(
        lattice=structure.lattice,
        species=species,
        coords=wrapped_cart_coords,
        coords_are_cartesian=True
    )
    
    print(f"✓ Wrapped {len(molecules_grouped)} CO₂ molecules")
    
    # Create supercell for better visualization (optional)
    supercell_matrix = [[2, 0, 0], [0, 2, 0], [0, 0, 2]]
    supercell_transform = SupercellTransformation(supercell_matrix)
    supercell_structure = supercell_transform.apply_transformation(wrapped_structure)
    
    # Create NGLView visualization
    view = nv.show_pymatgen(supercell_structure)
    view.clear_representations()
    
    # Add unit cell
    view.add_unitcell()
    
    if spacefill:
        # Add spacefill representation
        view.add_spacefill(radius_type='vdw', radius=0.4, color_scheme='element')
        # Also add thin bonds to show connectivity
        view.add_ball_and_stick(radius=0.1, bond_radius=0.05)
    else:
        # Add ball and stick with proper bond detection
        view.add_ball_and_stick(radius=0.3, bond_radius=0.1)
    
    # Add explicit C-O bonds to highlight molecular integrity
    view.add_distance(atom_pair=[['C', 'O']], 
                     label_visible=False, 
                     color='red',
                     radius=0.08)
    
    if show_axes:
        # Add coordinate axes
        view.shape.add_arrow([0, 0, 0], [2, 0, 0], [1, 0, 0], 0.3, "x-axis")
        view.shape.add_arrow([0, 0, 0], [0, 2, 0], [0, 1, 0], 0.3, "y-axis")
        view.shape.add_arrow([0, 0, 0], [0, 0, 2], [0, 0, 1], 0.3, "z-axis")
    
    view.camera = "perspective"
    
    # Print molecule information for debugging
    print(f"Molecule assignments: {len(set(mol for mol in molecule_assignment if mol is not None))} molecules found")
    
    return view, wrapped_structure, molecules_grouped

view1, wrapped_structure1, molecules1 = plot3d_with_wrapped_molecules(pa3, spacefill=False)
print("Supercell with wrapped molecules:")
display(view1)

## Your Turn: Initialize and Visualize Other CO₂ Space Groups

Now that you've seen how the Pa3 (cubic) structure works, it's time to explore the other CO₂ polymorphs! Each space group has different symmetry and requires different parameters. Work through each section step by step.

### Exercise 1: Cmce Space Group (Orthorhombic - Phase III)

The Cmce space group is orthorhombic, meaning it has three different lattice parameters: a, b, and c. It also requires both a bond length and a bond angle for proper initialization.

**Your task:** Complete the code below to initialize the Cmce structure.

**Hint:** Look at the function signature in `spacegroup.py` to see what parameters `Cmce` needs!

In [ ]:
# Cmce space group initialization
# TODO: Fill in the missing parameters

bond_length = ___  # Typical CO₂ bond length in Angstroms
a = ___           # Lattice parameter a (try around 5.0-6.0 Å)
b = ___           # Lattice parameter b (try around 4.5-5.5 Å) 
c = ___           # Lattice parameter c (try around 6.0-7.0 Å)
bond_angle = ___  # Bond angle in degrees (try around 30-60°)

# Initialize the Cmce structure
structure_cmce = Cmce(a=___, b=___, c=___)

# Adjust fractional coordinates with bond length and angle
structure_cmce.adjust_fractional_coords(___, ___)

# Build the unit cell
cmce_structure = build_unit_cell(structure_cmce)

print("Cmce structure built with:")
print(f"  Bond length: {bond_length} Å")
print(f"  Bond angle: {bond_angle}°") 
print(f"  Lattice constants: a={a}, b={b}, c={c} Å")

### Exercise 2: Visualize the Cmce Structure

**Your task:** Create a visualization function for the Cmce structure and display it.


In [ ]:
# TODO: Create visualization for Cmce structure



### Exercise 3: P42/mnm Space Group (Tetragonal - Phase II)

The P42/mnm space group is tetragonal, with two lattice parameters: a and c (where a = b ≠ c).

**Your task:** Initialize and visualize the P42/mnm structure.

In [ ]:
# P42/mnm space group initialization
# TODO: Complete the initialization

print("Initializing P42/mnm (tetragonal) structure...")

bond_length = ___  # Use the same bond length as before
a = ___           # Tetragonal a parameter (try around 5.0-6.0 Å)
c = ___           # Tetragonal c parameter (try around 6.0-8.0 Å)

# TODO: Initialize the P42mnm structure
structure_p42mnm = ___(a=___, c=___)

# TODO: Adjust fractional coordinates (hint: P42mnm only needs bond_length)
structure_p42mnm.adjust_fractional_coords(___)

# TODO: Build the unit cell
p42mnm_structure = ___

print("P42/mnm structure built with:")
print(f"  Bond length: {bond_length} Å")
print(f"  Lattice constants: a={a}, c={c} Å")

# TODO: Visualize the structure using your function from Exercise 2
# (You may need to modify it slightly for different space groups)

### Exercise 4: R-3c Space Group (Hexagonal - Phase VII)

The R-3c space group is the most complex, with hexagonal symmetry. It requires multiple bond lengths and angles.

**Your task:** Initialize the R-3c structure (this one is challenging!).

In [ ]:
# R-3c space group initialization
# TODO: This is the most complex space group - fill in carefully!

print("Initializing R-3c (hexagonal) structure...")

bond_length1 = ___    # First CO₂ bond length
bond_length2 = ___    # Second CO₂ bond length  
bond_angle_phi = ___  # Phi angle in degrees
bond_angle_theta = ___ # Theta angle in degrees
a = ___              # Hexagonal a parameter (try around 8-9 Å)
c = ___              # Hexagonal c parameter (try around 10-12 Å)

# TODO: Initialize the R3c structure
structure_r3c = ___(a=___, c=___)

# TODO: Adjust fractional coordinates with all four parameters
structure_r3c.adjust_fractional_coords(___, ___, ___, ___)

# TODO: Build the unit cell
r3c_structure = ___

print("R-3c structure built with:")
print(f"  Bond lengths: {bond_length1}, {bond_length2} Å")
print(f"  Bond angles: φ={bond_angle_phi}°, θ={bond_angle_theta}°")
print(f"  Lattice constants: a={a}, c={c} Å")

# TODO: Create visualization (this may be more challenging due to complexity)

### Exercise 5: Compare All Structures

**Your task:** Create a comparison of all the structures you've built.

In [ ]:
# Structure comparison
structures = {
    'Pa3 (Cubic)': pa3,
    'Cmce (Orthorhombic)': cmce_structure,
    'P42/mnm (Tetragonal)': p42mnm_structure,
    'R-3c (Hexagonal)': r3c_structure
}

print("Comparison of CO₂ Crystal Structures:")
print("=" * 50)

for name, structure in structures.items():
    print(f"\n{name}:")
    print(f"  Formula: {structure.composition.reduced_formula}")
    print(f"  Number of atoms: {len(structure)}")
    print(f"  Volume: {structure.volume:.2f} Å³")
    print(f"  Density: {structure.density:.3f} g/cm³")
    
    # TODO: Add lattice parameter information
    lattice = structure.lattice
    print(f"  Lattice: a={lattice.a:.3f}, b={lattice.b:.3f}, c={lattice.c:.3f} Å")
    print(f"  Angles: α={lattice.alpha:.1f}°, β={lattice.beta:.1f}°, γ={lattice.gamma:.1f}°")

# TODO: Create side-by-side visualizations of all structures

### Hints and Tips:

1. **For Cmce**: Remember it needs both `bond_length` and `bond_angle` parameters
2. **For P42/mnm**: This only needs `bond_length` (no angle)
3. **For R-3c**: This is the most complex - it needs `bond_length1`, `bond_length2`, `bond_angle_phi`, and `bond_angle_theta`
4. **Check the source code**: Look at `spacegroup.py` to see exactly what each class expects
5. **Start with reasonable values**: Use physical intuition for CO₂ bond lengths (~1.16 Å) and reasonable lattice parameters
6. **Test incrementally**: Get each structure working before moving to the next

### Expected Output:
When completed successfully, you should see:
- ✅ Four different CO₂ crystal structures initialized
- ✅ Interactive 3D visualizations showing different symmetries
- ✅ Proper molecular bonding (no broken CO₂ molecules)
- ✅ Comparison table showing structural differences

**Good luck! Work through each exercise step by step, and don't hesitate to experiment with different parameter values.**